In [23]:
!git clone https://github.com/Hotsnown/seminaire-bordeaux-2022.git seminaire &> /dev/null


# Classification Textuelle



### Importer les données

In [33]:
# lire les données dans un DataFrame Pandas
import pandas as pd
df = pd.read_csv("/content/seminaire/exercices/jour4/4.3 text classification/outcome_prediction.csv")

In [34]:
# print les 5 premières lignes de données
df.head()

,Unnamed: 0,content,role
0,0,Because CHAMPUS is not a party to this appeal ...,Complex
1,1,We affirm on all issues raised in the appeal a...,Affirm
2,2,"For the reasons that follow, we affirm.",Affirm
3,3,. We affirm.,Affirm
4,4,We reverse and remand for further proceedings ...,Reverse


Question : Peut-on prédire la nature des motifs ?

In [41]:
# définir X et y
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["content"])
y = [0 if y == "Reverse" else 1 for y in df["role"]]

In [42]:
# diviser X et y en ensembles d'entraînement et de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [43]:
# Entraînement d'un modèle de régression logistique sur l'ensemble d'entraînement.
from sklearn.linear_model import LogisticRegression

TODO : Créez un estimateur LogisticRegression, puis entraînez-le sur X_train et y_train
 

LogisticRegression()

In [44]:
# faire des prédictions de classe pour l'ensemble de test
y_pred_class = logreg.predict(X_test)

### Précision de la classification : pourcentage de prédictions correctes

In [45]:
# calculer la précision
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.904


Précision nulle : précision qui pourrait être obtenue en prédisant toujours la classe la plus fréquente.

In [49]:
# examiner la distribution des classes de l'ensemble de test (en utilisant une méthode Pandas Series)
y_test = pd.Series(y_test)

y_test.value_counts()

1    86
0    39
dtype: int64

In [50]:
# calculer le pourcentage de 1
y_test.mean()

0.688

In [51]:
# calculer le pourcentage de zéros
1 - y_test.mean()

0.31200000000000006

In [52]:
# calculer la précision nulle (pour les problèmes de classification binaire codés comme 0/1)
max(y_test.mean(), 1 - y_test.mean())

0.688

Comparaison des valeurs de réponse réelles et prédites

In [54]:
# imprimer les 25 premières réponses vraies et prédites
from __future__ import print_function
print('True:', y_test.values[0:25])
print('Pred:', y_pred_class[0:25])

True: [0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 0 0]
Pred: [0 1 0 1 1 1 0 1 0 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 0]


Conclusion :

* La précision de la classification est la mesure de classification la plus facile à comprendre.
* Mais, elle ne vous indique pas la distribution sous-jacente des valeurs de réponse.
* Et elle ne vous dit pas quels "types" d'erreurs votre classificateur commet.

### Matrice de confusion (confusion matrix)

Tableau qui décrit la performance d'un modèle de classification

In [55]:
# IMPORTANT : le premier argument est les valeurs réelles, le second argument est les valeurs prédites.
print(metrics.confusion_matrix(y_test, y_pred_class))

[[29 10]
 [ 2 84]]


Chaque observation de l'ensemble de test est représentée dans une seule case.
Il s'agit d'une matrice 2x2 car il y a deux classes de réponses.
Le format présenté ici n'est pas universel

Terminologie de base

* Vrais positifs (TP) : nous avons correctement prédit que le motif rejette la décision.
* Vrais négatifs (TN) : nous avons correctement prédit que le motif ne rejette pas la décision.
* Faux positifs (FP) : nous avons prédit à tort que le motif rejette la décision (une "erreur de type I").
* Faux négatifs (FN) : nous avons prédit à tort que le motif ne rejette pas la décision (une "erreur de type II").

In [56]:
# IMPORTANT : le premier argument est les valeurs réelles, le second argument est les valeurs prédites.
print('True:', y_test.values[0:25])
print('Pred:', y_pred_class[0:25])

True: [0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 0 0]
Pred: [0 1 0 1 1 1 0 1 0 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 0]


In [57]:
# IMPORTANT : le premier argument est les valeurs réelles, le second argument est les valeurs prédites.
confusion = metrics.confusion_matrix(y_test, y_pred_class)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

### Métriques calculées à partir d'une matrice de confusion

Précision de la classification : Globalement, combien de fois le classificateur est-il correct ?

In [58]:
print((TP + TN) / float(TP + TN + FP + FN))
print(metrics.accuracy_score(y_test, y_pred_class))

import seaborn as sns
sns.heatmap(confusion, annot=True)

0.904
0.904


Erreur de classification : Globalement, combien de fois le classificateur est-il incorrect ?

Également connu sous le nom de "taux de classification erronée".

In [59]:
print((FP + FN) / float(TP + TN + FP + FN))
print(1 - metrics.accuracy_score(y_test, y_pred_class))

0.096
0.09599999999999997


Sensibilité : Lorsque la valeur réelle est positive, combien de fois la prédiction est-elle correcte ?

Dans quelle mesure le classificateur est-il "sensible" à la détection d'instances positives ?
Également connu sous le nom de "taux de vrais positifs" ou "rappel".

In [60]:
print(TP / float(TP + FN))
print(metrics.recall_score(y_test, y_pred_class))

0.9767441860465116
0.9767441860465116


Spécificité : Lorsque la valeur réelle est négative, combien de fois la prédiction est-elle correcte ?

Dans quelle mesure le classificateur est-il "spécifique" (ou "sélectif") pour prédire des instances positives ?

In [61]:
print(TN / float(TN + FP))

0.7435897435897436


Taux de faux positifs : Lorsque la valeur réelle est négative, combien de fois la prédiction est-elle incorrecte ?

In [62]:
print(FP / float(TN + FP))

0.2564102564102564


La précision : Lorsqu'une valeur positive est prédite, combien de fois la prédiction est-elle correcte ?

Quelle est la "précision" du classifieur lorsqu'il prédit des instances positives ?

In [63]:
print(TP / float(TP + FP))
print(metrics.precision_score(y_test, y_pred_class))

0.8936170212765957
0.8936170212765957


Conclusion :

La matrice de confusion vous donne une image plus complète de la performance de votre classificateur.
Elle vous permet également de calculer diverses métriques de classification, et ces métriques peuvent guider votre sélection de modèle.

1. Jour 1
    * Variables
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.1%20helloworld.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.2%20%C3%A9viter%20les%20errreurs%20de%20nommage.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.3%20mini-project.ipynb#scrollTo=RPB2xCMdA6lV)
    * Strings
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.1%20concat.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.2%20string_methods.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.3%20mini-project.ipynb)
    * Opérations
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.1%20math.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.2%20bool%C3%A9en.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.3%20mini-project.ipynb)

2. Jour 2
    * Listes
        * [Définition](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.1%20d%C3%A9finition%20liste.ipynb?hl=fr)
        * [Strings et listes](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.2%20String%20as%20list%20of%20characters.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.3%20mini-project.ipynb?hl=fr)
    * Fonctions
        * [Définition I](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.1%20d%C3%A9finition%20fonctions.ipynb?hl=fr)
        * [Définition II](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.2%20scope%20et%20fonctions%20imbriqu%C3%A9es.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.3%20mini-project.ipynb?hl=fr) 
    * Librairies
        * [Pandas](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.1%20Pandas.ipynb?hl=fr)
        * [Matplotlib](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.2%20Matplotlib.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.3%20mini-project.ipynb?hl=fr) 
3. Jour 3
    * Introduction à la NLP
        * [Charger des un corpus](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.1%20Accessing%20Text.ipynb?hl=fr)
        * [Traitement de texte dans Pandas](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.2%20Working%20with%20text%20data%20in%20pandas.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.3%20mini-project.ipynb?hl=fr)
    * Segmentation
        * [Segmentation de tokens](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.1%20Token%20segmentation.ipynb?hl=fr)
        * [Segmentation de phrase](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.2%20Sentence%20segmentation.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.3%20mini-project.ipynb?hl=fr)
    * Nettoyage de texte
        * [Stopwords](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.1%20stopwords.ipynb?hl=fr)
        * [Normalisation](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.2%20Normalizing%20Text.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.3%20mini-project.ipynb?hl=fr)
4. Jour 4
    * Apprentissage supervisé
        * [Régression linéaire](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.1%20linear%20regression.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.3%20mini-project.ipynb?hl=fr)
    * Pré-traitement de texte
        * [Featurization de textes](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.1%20text%20featurization.ipynb?hl=fr)
        * [Featurization de labels](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.2%20label%20featurization.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.3%20mini-project.ipynb?hl=fr)
    * Classification de texte
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.3%20mini-project.ipynb?hl=fr)
5. Jour 5
    * [Projet final](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour5/final-project.ipynb?hl=fr)